In [1]:
import pandas as pd


# Model training

In [2]:
df=pd.read_csv("gemstone.csv")
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [3]:
df=df.drop(['id'],axis=1)

In [4]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
## independent and dependent features
X=df.drop(['price'],axis=1)
y=df.price

In [6]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [7]:
y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

In [8]:
### define the categorical columns and numerical columns
categorical_columns = X.select_dtypes(include='object').columns
numerical_columns = X.select_dtypes(exclude='object').columns

In [9]:
categorical_columns

Index(['cut', 'color', 'clarity'], dtype='object')

In [10]:
numerical_columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [19]:
## define the the custom ranking for each ordinal veriables (i.e categorical feactures)
cut_categorie = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['D','E','F','G','H','I','J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [17]:
from sklearn.impute import SimpleImputer  ## Handling Missing Values
from sklearn.preprocessing import StandardScaler  ## Handling Feature Scalling
from sklearn.preprocessing import OrdinalEncoder  ## Ordinal Encoder
## pipe lines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [27]:
## Numericali Pipeline
num_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='median')),
        ('scaler',StandardScaler())
    ]
)

## categorical Pipeline
cat_pipeline = Pipeline(
    steps =[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencode',OrdinalEncoder(categories=[cut_categorie,color_categories,clarity_categories])),
        ('scaler',StandardScaler()) 
    ]
)

preprocessor=ColumnTransformer([
    ('nem_pipeline',num_pipeline,numerical_columns),
('cat_pipeline',cat_pipeline,categorical_columns)
])

In [30]:
## Trsin test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=32)


In [36]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [37]:
X_train.head()

,nem_pipeline__carat,nem_pipeline__depth,nem_pipeline__table,nem_pipeline__x,nem_pipeline__y,nem_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.888807,-0.851198,1.441566,-0.944040,-0.927206,-0.993419,-0.129364,1.529933,0.685799
1,0.473470,1.181202,-1.163589,0.589495,0.616053,0.703190,-2.137944,-0.935041,0.019230
2,-0.607702,-0.574052,-0.642558,-0.502021,-0.491462,-0.529389,0.874927,0.297446,1.352368
3,2.852048,0.072620,-1.163589,2.321487,2.277325,2.283791,0.874927,2.146176,0.019230
4,0.041001,-1.497871,1.441566,0.318871,0.271089,0.166655,-0.129364,-0.318798,0.685799


In [38]:
X_test.head()

,nem_pipeline__carat,nem_pipeline__depth,nem_pipeline__table,nem_pipeline__x,nem_pipeline__y,nem_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.083417,-0.389289,0.920535,-1.313892,-1.308482,-1.326941,-1.133654,-1.551285,0.685799
1,-0.196857,-1.867398,1.441566,0.030206,0.107685,-0.079861,-1.133654,1.529933,-1.313909
2,-1.040171,-0.851198,-0.121527,-1.196622,-1.190468,-1.239935,0.874927,-1.551285,0.685799
3,-0.153610,-0.574052,-0.121527,0.057268,0.080451,0.007145,0.874927,0.913689,1.352368
4,0.495093,0.719293,0.399504,0.643620,0.597897,0.688689,-0.129364,-1.551285,-1.313909


In [39]:
X_train.shape,X_test.shape

((135501, 9), (58072, 9))

In [40]:
### Model Trining
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [41]:
regression=LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [43]:
regression.coef_

array([ 6432.15885064,  -130.04060851,   -64.75184961, -1905.11295757,
        -292.6220071 ,   -79.7619032 ,    74.41515792,  -465.98515078,
         652.73563718])

In [45]:
regression.intercept

3970.0849809226497

In [52]:
import numpy as np
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2_squared=r2_score(true,predicted)
    return mae,mse,rmse,r2_squared

In [58]:
# Model Training
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
model_list=list()
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    ## Make prediction
    y_pred=model.predict(X_test)

    mae, mse, rmse, r2_squared = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training performence')
    print('MSE: ',mse)
    print('MAE: ',mae)
    print('RMSE: ',rmse)
    print('R2_score: ',r2_squared*100)
    print("******************************")
    print()



LinearRegression
Model Training performence
MSE:  1021471.2631449203
MAE:  677.202109489184
RMSE:  1010.6786151615756
R2_score:  93.71701066078352
******************************

Lasso
Model Training performence
MSE:  1021929.2436779373
MAE:  678.40351049175
RMSE:  1010.9051605753813
R2_score:  93.71419365857274
******************************

Ridge
Model Training performence
MSE:  1021459.9475028478
MAE:  677.2308920416484
RMSE:  1010.6730171043688
R2_score:  93.71708026240725
******************************

ElasticNet
Model Training performence
MSE:  2328870.375443008
MAE:  1058.8702566849374
RMSE:  1526.0636865619363
R2_score:  85.67530162691462
******************************

